**A) Data Preprocessing **





In [1]:
import re

# Sample data
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# 1. Lowercase the sentences
data = [(en.lower(), fr.lower()) for en, fr in data]

# 2. Tokenize the sentences by splitting each string
tokenized_data = [(en.split(), fr.split()) for en, fr in data]

# Display the tokenized pairs
for en, fr in tokenized_data:
    print(f"English: {en} -> French: {fr}")


English: ['hello'] -> French: ['bonjour']
English: ['how', 'are', 'you'] -> French: ['comment', 'ça', 'va']
English: ['i', 'am', 'fine'] -> French: ['je', 'vais', 'bien']
English: ['what', 'is', 'your', 'name'] -> French: ['comment', 'tu', "t'appelles"]
English: ['my', 'name', 'is'] -> French: ['je', "m'appelle"]
English: ['thank', 'you'] -> French: ['merci']
English: ['goodbye'] -> French: ['au', 'revoir']


**B) Build Seq2Seq Model**

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample data
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# Prepare English and French sentences
english_sentences = [pair[0] for pair in data]
french_sentences = [pair[1] for pair in data]

# Tokenization and integer encoding
def tokenize_and_pad(sentences, max_length):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    sequences = pad_sequences(sequences, maxlen=max_length, padding="post")
    return sequences, tokenizer

# Define max length for padding
max_len_en = max(len(s.split()) for s in english_sentences)
max_len_fr = max(len(s.split()) for s in french_sentences)

# Tokenize and pad English and French sentences
input_sequences, input_tokenizer = tokenize_and_pad(english_sentences, max_len_en)
target_sequences, target_tokenizer = tokenize_and_pad(french_sentences, max_len_fr)

# Get vocabulary sizes
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

# Convert target sentences to one-hot encoding for output layer
target_data = tf.keras.utils.to_categorical(target_sequences, target_vocab_size)

# Model hyperparameters
embedding_dim = 256
latent_dim = 512

# Encoder
encoder_inputs = Input(shape=(max_len_en,))
encoder_embedding = tf.keras.layers.Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len_fr,))
decoder_embedding = tf.keras.layers.Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that takes encoder and decoder inputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit([input_sequences, target_sequences], target_data, batch_size=16, epochs=100)

# Summary of model architecture
model.summary()


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0952 - loss: 2.6392
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.3333 - loss: 2.5860
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.3333 - loss: 2.5267
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.3333 - loss: 2.4514
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.3333 - loss: 2.3496
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.3333 - loss: 2.2132
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.3333 - loss: 2.0621
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.3333 - loss: 1.9954
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.3333 - loss: 2.0057
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.3810 - loss: 1.9446
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.3810 - loss: 1.8363
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 256)         │          3,840 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 3, 256)         │          3,584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 512), (None,   │      1,574,912 │ embedding[0][0]        │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 3, 512),       │      1,574,912 │ embedding_1[0][0],     │
│                           │ (None, 512), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 512)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 3, 14)          │          7,182 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,493,292 (36.21 MB)

 Trainable params: 3,164,430 (12.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,328,862 (24.14 MB)

**C)Preparing the Data for Training**

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample dataset
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# Split data into English and French sentences
english_sentences = [pair[0].lower() for pair in data]
french_sentences = [pair[1].lower() for pair in data]

# Initialize tokenizers for English and French
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

# Fit tokenizers on the sentences to learn the vocabulary
english_tokenizer.fit_on_texts(english_sentences)
french_tokenizer.fit_on_texts(french_sentences)

# Convert sentences to sequences of integers
input_sequences = english_tokenizer.texts_to_sequences(english_sentences)
target_sequences = french_tokenizer.texts_to_sequences(french_sentences)

# Define maximum sequence lengths for padding
max_len_en = max(len(seq) for seq in input_sequences)
max_len_fr = max(len(seq) for seq in target_sequences)

# Pad sequences to make them all the same length
input_sequences = pad_sequences(input_sequences, maxlen=max_len_en, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_len_fr, padding='post')

# Vocabulary sizes
input_vocab_size = len(english_tokenizer.word_index) + 1  # +1 for padding token
target_vocab_size = len(french_tokenizer.word_index) + 1

# Prepare target data for training
# Decoder target data is shifted by one position
target_data = np.zeros((len(target_sequences), max_len_fr, target_vocab_size), dtype="float32")

for i, seq in enumerate(target_sequences):
    for t, word_id in enumerate(seq):
        if word_id > 0:
            target_data[i, t, word_id] = 1.0  # One-hot encoding for each word in sequence

print("Input sequences (English):")
print(input_sequences)
print("Target sequences (French):")
print(target_sequences)
print("Target data (One-hot encoded for decoder):")
print(target_data.shape)


Input sequences (English):
[[ 4  0  0  0]
 [ 5  6  1  0]
 [ 7  8  9  0]
 [10  2 11  3]
 [12  3  2  0]
 [13  1  0  0]
 [14  0  0  0]]
Target sequences (French):
[[ 3  0  0]
 [ 1  4  5]
 [ 2  6  7]
 [ 1  8  9]
 [ 2 10  0]
 [11  0  0]
 [12 13  0]]
Target data (One-hot encoded for decoder):
(7, 3, 14)


**D)Train the model on the dataset**

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Sample data
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

# Split data into English and French sentences
english_sentences = [pair[0].lower() for pair in data]
french_sentences = [pair[1].lower() for pair in data]

# Tokenizers
english_tokenizer = Tokenizer()
french_tokenizer = Tokenizer()

# Fit tokenizers on sentences
english_tokenizer.fit_on_texts(english_sentences)
french_tokenizer.fit_on_texts(french_sentences)

# Convert sentences to sequences of integers
input_sequences = english_tokenizer.texts_to_sequences(english_sentences)
target_sequences = french_tokenizer.texts_to_sequences(french_sentences)

# Define max sequence lengths
max_len_en = max(len(seq) for seq in input_sequences)
max_len_fr = max(len(seq) for seq in target_sequences)

# Pad sequences
input_sequences = pad_sequences(input_sequences, maxlen=max_len_en, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_len_fr, padding='post')

# Vocabulary sizes
input_vocab_size = len(english_tokenizer.word_index) + 1
target_vocab_size = len(french_tokenizer.word_index) + 1

# One-hot encode target sequences
target_data = np.zeros((len(target_sequences), max_len_fr, target_vocab_size), dtype="float32")
for i, seq in enumerate(target_sequences):
    for t, word_id in enumerate(seq):
        if word_id > 0:
            target_data[i, t, word_id] = 1.0

# Model hyperparameters
embedding_dim = 256
latent_dim = 512

# Encoder
encoder_inputs = Input(shape=(max_len_en,))
encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_len_fr,))
decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit([input_sequences, target_sequences], target_data, batch_size=16, epochs=300)

# Summary of model
model.summary()


Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 1.8880
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.5238 - loss: 1.8637
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5238 - loss: 1.8384
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.5238 - loss: 1.8108
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.4762 - loss: 1.7793
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.4762 - loss: 1.7422
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.4762 - loss: 1.6975
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.4286 - loss: 1.6432
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.4286 - loss: 1.5772
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.3810 - loss: 1.4988
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.4286 - loss: 1.4099
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 4, 256)         │          3,840 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 3, 256)         │          3,584 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 512), (None,   │      1,574,912 │ embedding_2[0][0]      │
│                           │ 512), (None, 512)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, 3, 512),       │      1,574,912 │ embedding_3[0][0],     │
│                           │ (None, 512), (None,    │                │ lstm_2[0][1],          │
│                           │ 512)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 3, 14)          │          7,182 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,493,292 (36.21 MB)

 Trainable params: 3,164,430 (12.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,328,862 (24.14 MB)

**E) Inference Setup for Translation**

In [8]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Sample data and preprocessing steps here (assuming you have `english_tokenizer` and `french_tokenizer` set up)

# Reverse-lookup token index to decode sequences back to words
reverse_input_word_index = {v: k for k, v in english_tokenizer.word_index.items()}
reverse_target_word_index = {v: k for k, v in french_tokenizer.word_index.items()}

# Encoder model (assuming `encoder_inputs`, `encoder_states` are defined from training setup)
encoder_model = Model(encoder_inputs, encoder_states)

# Inference setup for the decoder
latent_dim = 512  # Same as training latent_dim
embedding_dim = 256  # Same as training embedding_dim

decoder_embedding_inference = Embedding(input_dim=target_vocab_size, output_dim=embedding_dim)
decoder_inputs_single = Input(shape=(1,))
embedded_decoder_input = decoder_embedding_inference(decoder_inputs_single)

# Define decoder states and layers
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    embedded_decoder_input, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# Define the inference decoder model
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Translation function
def translate_sentence(input_text):
    input_seq = english_tokenizer.texts_to_sequences([input_text.lower()])
    input_seq = pad_sequences(input_seq, maxlen=max_len_en, padding='post')
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = french_tokenizer.word_index.get('<start>', 1)

    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index.get(sampled_token_index, '')

        if sampled_word == '<end>' or len(decoded_sentence) > max_len_fr:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return ' '.join(decoded_sentence)

# Test translation
input_text = "hello"
translation = translate_sentence(input_text)
print(f"Input: {input_text}")
print(f"Translation: {translation}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Input: hello
Translation: bonjour revoir revoir revoir


**F) Translate New Sentences**

In [9]:
# Define new sentences to translate
new_sentences = [
    "good morning",
    "how are you doing",
    "thank you very much",
    "I am learning French",
    "see you tomorrow"
]

# Translate each sentence and print the result
for sentence in new_sentences:
    translation = translate_sentence(sentence)
    print(f"Input: {sentence}")
    print(f"Translation: {translation}")
    print("-" * 30)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Input: good morning
Translation: bonjour revoir revoir revoir
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Input: how are you doing
Translation: comment ça va va
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Input: thank you very much
Translation: merci revoir revoir revoir
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1

**G) Experimenting and Improving the Model by large dataset and hyper tune parameter.**

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample sentences for the dataset (for illustration)
english_sentences = [
    "hello",
    "how are you",
    "I am fine",
    "what is your name",
    "my name is",
    "thank you",
    "goodbye"
]

french_sentences = [
    "bonjour",
    "comment ça va",
    "je vais bien",
    "comment tu t'appelles",
    "je m'appelle",
    "merci",
    "au revoir"
]

# Tokenization
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(english_sentences)
english_vocab_size = len(english_tokenizer.word_index) + 1  # +1 for padding
source_vocab_size = english_vocab_size  # This is what we need

french_tokenizer = Tokenizer()
french_tokenizer.fit_on_texts(french_sentences)
french_vocab_size = len(french_tokenizer.word_index) + 1  # +1 for padding
target_vocab_size = french_vocab_size  # This is what we need

# Maximum sequence lengths (define based on your dataset)
max_len_en = max(len(sentence.split()) for sentence in english_sentences)  # Example for English
max_len_fr = max(len(sentence.split()) for sentence in french_sentences)  # Example for French

# Example encoding
encoder_input_data = english_tokenizer.texts_to_sequences(english_sentences)
decoder_input_data = french_tokenizer.texts_to_sequences(french_sentences)

# Padding sequences
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_len_en, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_len_fr, padding='post')

# Now we can define the model
embedding_dim = 256  # Example dimension for embedding
latent_dim = 512     # Example latent dimension for LSTM
batch_size = 64      # Example batch size
epochs = 20          # Example number of epochs

# Define input and embedding layers
encoder_inputs = Input(shape=(max_len_en,))
encoder_embedding = Embedding(input_dim=source_vocab_size, output_dim=embedding_dim)(encoder_inputs)

# Use a Bidirectional LSTM
encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True, dropout=0.2))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

# Concatenate forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Define the decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=target_vocab_size, output_dim=embedding_dim)(decoder_inputs)

decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True, dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

# Attention mechanism
attention = tf.keras.layers.Attention()
context_vector = attention([decoder_outputs, encoder_outputs])

# Concatenate context vector with decoder outputs
decoder_combined_context = Concatenate()([context_vector, decoder_outputs])

# Define dense layers for prediction
output_dense = TimeDistributed(Dense(target_vocab_size, activation="softmax"))
decoder_outputs = output_dense(decoder_combined_context)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit the model with early stopping (dummy data used here for illustration)
# You should use your actual encoder_input_data and decoder_input_data
# history = model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2,
#     callbacks=[early_stopping]
# )

# Note: Uncomment the fitting code and ensure decoder_target_data is defined properly.
